In [65]:
from normalize_data import getNormalizedDataset
import pandas as pd

In [66]:
df_train = pd.read_csv('train.csv',
        index_col='id',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category'
            },
        parse_dates=['fecha'])
pd.set_option('display.float_format', '{:.2f}'.format)

df_train = getNormalizedDataset(df_train)

In [67]:
df_train.head()

,titulo,descripcion,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,precio_m2,extras
id,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Benito Juárez,Distrito Federal,0.00,2.00,1.00,2.00,80.00,80.00,2015-08-23,0,0,0,0,0,2273000.00,28412.50,1.00
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,268.00,2013-06-28,0,0,0,1,1,3600000.00,20000.00,2.00
247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Tonalá,Jalisco,5.00,3.00,2.00,2.00,144.00,166.00,2015-10-17,0,0,0,0,0,1200000.00,7228.92,2.00
209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,Zinacantepec,Edo. de México,1.00,2.00,1.00,1.00,63.00,67.00,2012-03-09,0,0,0,1,1,650000.00,9701.49,1.00
185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,Zapopan,Jalisco,10.00,2.00,1.00,1.00,95.00,95.00,2016-06-07,0,0,0,0,0,1150000.00,12105.26,1.00


# Preprocesamiento

In [68]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [69]:
def preprocessing(df):
    # Saco titulo, descripcion, precio_m2, precio_dolar, extras
    df['dia'] = df_train.fecha.dt.day
    df['mes'] = df_train.fecha.dt.month
    df['año'] = df_train.fecha.dt.year
    
    df.drop(['titulo','descripcion', 'fecha'], axis=1, inplace=True)
    
    # Label encoding
    cat_features = ['tipodepropiedad', 'ciudad', 'provincia']
    encoder = LabelEncoder()
    encoded = df[cat_features].apply(encoder.fit_transform)

    df.drop(['tipodepropiedad','provincia','ciudad'], axis=1, inplace=True)
    data_cols = list(df.columns)
    baseline_data = df[data_cols].join(encoded)

In [70]:
baseline_data = preprocessing(df_train)

In [71]:
baseline_data

In [72]:
features = baseline_data[['tipodepropiedad', 'ciudad', 'provincia',
       'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'dia', 'mes', 'año']]

label = baseline_data['precio']

#Los nan eran de antiguedad
features.fillna(0, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.25, random_state=1)

TypeError: 'NoneType' object is not subscriptable

# Modelo

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train, y_train)
# Primero tengo que pasar todos los labels a numericos.

In [ ]:
# los coeficientes dan idea de cuanto crece el precio al aumentar 1 de cada uno de los features.
cdf = pd.DataFrame(lm.coef_, features.columns, columns=['Coeff'])
cdf

# Predicciones

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Comparo predicciones con lo que de verdad es
plt.scatter(y_test, predictions)

In [ ]:
sns.distplot((y_test-predictions))
# Las diferencias entre las predicciones y la realidad son bastante distribuidas. 
# Tiende a haber muchos con diferencia cercana a 0

# Metricas

In [ ]:
from sklearn import metrics
import numpy as np

def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

# La razón para usar error logarítmico es que relativiza el error al valor absoluto considerado. Por ejemplo, consideremos un error absoluto de 1000 sobre un valor absoluto de 10 y uno de 100.000:

In [ ]:
actual_1 = 10
actual_2 = 1000000
error = 1000

abs_error_1 = np.abs(actual_1 - (actual_1 + error))
abs_error_2 = np.abs(actual_2 - (actual_2 + error))

log_error_1 = np.abs(np.log(actual_1 + 1) - np.log(actual_1 + error + 1))
log_error_2 = np.abs(np.log(actual_2 + 1) - np.log(actual_2 + error + 1))

print(f"Error relativo grande - Abs: {abs_error_1:.4f}, Log:{log_error_1:.4f}")
print(f"Error relativo chico  - Abs: {abs_error_2:.4f}, Log:{log_error_2:.4f}")

In [ ]:
linear_rmsle = RMSLE(y_test, predictions)
linear_rmsle_train = RMSLE(y_train, lm.predict(X_train))
print(f"RMSLE Linear Regression (train): {linear_rmsle_train:.5f}")
print(f"RMSLE Linear Regression: {linear_rmsle:.5f}")

In [ ]:
#RMSE, sin logaritmo
np.sqrt(metrics.mean_squared_error(y_test, predictions))

In [ ]:
#Mean absolute error (el que importa para la competencia)
metrics.mean_absolute_error(y_test, predictions)

# Submit

In [ ]:
df_submit = pd.read_csv('test.csv',
        index_col='id',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category'
            },
        parse_dates=['fecha'])


In [ ]:
df_submit = getNormalizedDataset(df_submit, 'test')
df_submit.head()

In [ ]:
df_submit = preproccesing(df_submit)

linear_pred = lm.predict(df_submit)

res = pd.DataFrame(linear_pred, index=df_submit.index, columns=['precio'])
display(res.head())
res.to_csv("submission.csv", header=False) # RMSLE=0.65487